In [1]:
import pandas as pd
from gerrychain import Graph
from networkx import DiGraph
import gurobipy as gp
from gurobipy import GRB
import math



In [2]:
def read_matrix(filename):
    df = pd.read_csv(filename, index_col=0)
    edges = []
    
    for i, row in df.iterrows():
        for j, value in row.items():
            if value != 0:
                edges.append((int(i), int(j), int(value)))

    return edges

In [3]:
def read_population(filename):
    with open(filename, 'r') as file:
        total_population = int(file.readline().strip().split('=')[-1])
        data = []
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                identifier, population = int(parts[0]), int(parts[1])
                data.append((identifier, population))
            else:
                print(f"Unexpected line format: {line}")
    return total_population, data

In [5]:
RI = 2
edges = read_matrix('data/RI_distances.csv')
#print(edges)
total_population, population = read_population('data/RI.population')
G = Graph()
G.add_weighted_edges_from(edges)
for (node, pop) in population:
    #print(node)
    G.nodes[node]['pop'] = pop
    G.nodes[node]['name'] = node

# DG = DiGraph(G)



In [90]:
#weight = G.get_edge_data(0, 1)['weight']


In [125]:
#weight = G.get_edge_data(0, 1)['population']
#print(G.nodes[0]['population'])
m = gp.Model()
deviation = 0.02
k = 2
x = m.addVars( G.nodes, G.nodes, vtype=GRB.BINARY )
L = math.ceil( ( 1 - deviation / 2 ) * total_population / k )
U = math.floor( ( 1 + deviation / 2 ) * total_population / k )



In [126]:
print( G.get_edge_data(0, 0))

None


In [127]:
dist = { (i,j) : 0 for i in G.nodes for j in G.nodes }
for i in G.nodes:
    for j in G.nodes:
        #print("*")
        #print( G.get_edge_data(i, j)['weight'])
        if G.get_edge_data(i, j) is not None:
            dist[i,j] = G.get_edge_data(i, j)['weight']
        
#print(dist)
#dist.values()
    

In [115]:
# add constraints saying that each county i is assigned to one district


In [128]:
m.setObjective( gp.quicksum( dist[i,j] * dist[i,j] * G.nodes[i]['pop'] * x[i,j] for i in G.nodes for j in G.nodes ), GRB.MINIMIZE )
m.addConstrs( gp.quicksum( x[i,j] for j in G.nodes ) == 1 for i in G.nodes )
m.addConstr( gp.quicksum( x[j,j] for j in G.nodes ) == k )

# add constraints that say: if j roots a district, then its population is between L and U.
m.addConstrs( gp.quicksum( G.nodes[i]['pop'] * x[i,j] for i in G.nodes ) >= L * x[j,j] for j in G.nodes )
m.addConstrs( gp.quicksum( G.nodes[i]['pop'] * x[i,j] for i in G.nodes ) <= U * x[j,j] for j in G.nodes )
# add coupling constraints saying that if i is assigned to j, then j is a center.
m.addConstrs( x[i,j] <= x[j,j] for i in G.nodes for j in G.nodes )

m.update()

In [129]:
# solve, making sure to set a 0.00% MIP gap tolerance
m.Params.MIPGap = 0.0

m.optimize()

Set parameter MIPGap to value 0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.4.0 21E230)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 63251 rows, 62500 columns and 310258 nonzeros
Model fingerprint: 0xd17439f3
Variable types: 0 continuous, 62500 integer (62500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e+03, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 250 rows and 0 columns
Presolve time: 0.66s
Presolved: 63001 rows, 62500 columns, 310258 nonzeros
Variable types: 0 continuous, 62500 integer (62500 binary)
Found heuristic solution: objective 3.823956e+08
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.11s

Solved with dual simplex

Root relaxation: objective 2.374514e+08, 26530 iterations, 3.87 seconds (8.13 work units)

   

In [130]:
centers = [ j for j in G.nodes if x[j,j].x > 0.5 ]

districts = [ [ i for i in G.nodes if x[i,j].x > 0.5 ] for j in centers ]
district_populations = [ sum(G.nodes[i]["pop"] for i in districts[j]) for j in range(k) ]
district_counties = [ [ G.nodes[i]['name'] for i in districts[j] ] for j in range(k)]




In [131]:
print("Centers: ",centers)
print("Districts: ",districts)
print("District pops" ,district_populations)
print("District counties: ",district_counties)


Centers:  [73, 93]
Districts:  [[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 69, 70, 71, 72, 73, 74, 75, 76, 77, 83, 84, 85, 86, 87, 89, 96, 97, 98, 114, 115, 121, 129, 138, 161, 162, 168, 170, 172, 181, 188, 197, 198, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 243], [7, 62, 63, 64, 65, 66, 67, 68, 78, 79, 80, 81, 82, 88, 90, 91, 92, 93, 94, 95, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 163, 164, 165, 166, 167, 169, 171, 173, 174, 175, 176, 177, 178, 179, 

In [132]:
for i in range(0,k):
    print(f"district {i} contains counties {district_counties[i]} with pop average {district_populations[i]}")

district 0 contains counties [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 69, 70, 71, 72, 73, 74, 75, 76, 77, 83, 84, 85, 86, 87, 89, 96, 97, 98, 114, 115, 121, 129, 138, 161, 162, 168, 170, 172, 181, 188, 197, 198, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 243] with pop average 543446
district 1 contains counties [7, 62, 63, 64, 65, 66, 67, 68, 78, 79, 80, 81, 82, 88, 90, 91, 92, 93, 94, 95, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 163, 164, 165, 166, 1